# _**Database Creator**_

### _Imports_

In [1]:
import json
from requests import Session
from time import sleep

SESSION = Session()

### _Functions_

In [2]:
def getUsefulData (game : dict):
    if 'dlc' in game:
        game['dlc'] = len(game['dlc'])
    if 'achievements' in game:
        game['achievements'] = game['achievements']['total']
    if 'metacritic' in game:
        game['metacritic'] = game['metacritic']['score']
    game['release_date'] = game['release_date']['date']
    if 'recommendations' in game:
        game['recommendations'] = game['recommendations']['total']
    return {key: game[key] for key in game.keys() & {'required_age', 'pc_requirements', 'mac_requirements',
                                                     'linux_requirements', 'developers', 'publishers', 'price_overview',
                                                     'platforms', 'metacritic', 'categories', 'genres', 'recommendations',
                                                     'release_date', 'ratings'
                                                    }}

## **<u>Get all steam games</u>**

In [ ]:
response = SESSION.get("https://api.steampowered.com/ISteamApps/GetAppList/v2/")
assert response.status_code == 200
steamGames = json.loads(response.text)
steamGames['applist']['apps'] = list(filter(lambda d:  not (d['name'] == "" or d['name'].endswith("Playtest") or
                                                            d['name'].endswith("Demo")) or d['name'].upper().__contains__("DLC") or
                                                            d['name'].upper().__contains__("EXPANSION PA") or
                                                            d['name'].upper().__contains__("SOUNDTRACK"),
                                            steamGames['applist']['apps']))

with open('steamGames.json', "w") as steamGamesFile:
    steamGamesFile.seek(0)
    json.dump(steamGames, steamGamesFile)
    steamGamesFile.truncate()

## **<u>Get steam games owner quantity</u>**

In [ ]:
page = 0
data = []

while True:
    response = SESSION.get("https://steamspy.com/api.php?request=all&page=" + str(page))
    if response.status_code != 200:
        break

    x = list(map(lambda d: {key: d[key] for key in d.keys() & {'appid', 'name', 'owners', 'average_forever',
                                                               'average_2weeks', 'median_forever', 'median_2weeks'}},
                 list(json.loads(response.text).values())))
    data = data + x
    page += 1

with open('steamGamesOwner.json', "w") as steamGamesFile:
    json.dump(data, steamGamesFile)
    steamGamesFile.truncate()

## **<u>Get games details</u>**

In [4]:
with open("steamGames.json", "r+") as gamesList:
    applist = json.load(gamesList)
    games : list = applist['applist']['apps']
    i = 0
    while i < len(games):
        if len(games[i]) > 2:
            i += 1
            continue
        try:
            response = SESSION.get("https://store.steampowered.com/api/appdetails?appids=" + str(games[i]['appid']))
            # response = SESSION.get("https://store.steampowered.com/api/appdetails?appids=" + str(2407760))
            while not response.status_code == 200:
                print(games[i]['appid'])
                print(response.status_code)
                sleep(60)
                response = SESSION.get("https://store.steampowered.com/api/appdetails?appids=" + str(games[i]['appid']))
                if response.status_code == 200:
                    print('Aquired !')
            x = json.loads(response.text)[str(games[i]['appid'])]
            if x['success'] == True:
                x = x['data']
                if x['type'] == 'game':
                    if 'dlc' in x and len(x['dlc']) > 0:
                        games = list(filter(lambda d: not d["appid"] in x['dlc'], games))
                    games[i] = games[i] | getUsefulData(x)
                else:
                    games.pop(i)
                    i -= 1
            else:
                    games.pop(i)
                    i -= 1
            i += 1
        except (Exception, BaseException) as err:
            print(games[i]['appid'])
            print(response.status_code)
            applist['applist']['apps'] = games
            gamesList.seek(0)
            json.dump(applist, gamesList)
            gamesList.truncate()
            raise err
    
    applist['applist']['apps'] = games
    gamesList.seek(0)
    json.dump(applist, gamesList)
    gamesList.truncate()

1546190
200


KeyError: '1546190'

## **<u>Merge the files</u>**